In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/james/anaconda3/envs/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# torch.manual_seed(123)

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {str(DEVICE).upper()}")

Device: CUDA


In [4]:
# MODEL_DIR = "/mnt/artemis/library/weights/meta/llama-2/7Bf/hf"
MODEL_DIR = "/mnt/artemis/library/weights/mistral/OpenHermes-2-Mistral-7B"

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype = torch.float16,
    device_map="auto",
    )

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.40s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
print(tokenizer.bos_token)
print(tokenizer.eos_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s>
<|im_end|>


In [7]:
en1 = "Prometheus stole fire from the gods and gave it to man."
# en1 = "What is 2+2=?"
tok = tokenizer(en1, return_tensors='pt').to(DEVICE).input_ids
print(tok)
print(tok.shape)

tensor([[    1, 12948, 13334,   381, 27613,  3339,   477,   272, 16578,   304,
          3395,   378,   298,   676, 28723]], device='cuda:0')
torch.Size([1, 15])


In [8]:
attn_mask = torch.tensor([1] * len(tok[0])).unsqueeze(0).to(DEVICE)
print(attn_mask.shape)

tok = model.generate(
    input_ids = tok,
    attention_mask = attn_mask,
    max_new_tokens = 512,
    do_sample = True,
    top_p = 1.0,
    top_k = 10,
    temperature = 0.7,
    pad_token_id = tokenizer.pad_token_id,
    repetition_penalty = 1.0,
    length_penalty = 1.0,
    )

torch.Size([1, 15])


In [9]:
response = tokenizer.batch_decode(
    sequences = tok,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
    padding_side='left')[0]

for word in response.strip():
    print(word, end='', flush=True)

Prometheus stole fire from the gods and gave it to man. In return he was punished by being chained to a rock where an eagle would eat his liver each day. His liver would grow back, so the eagle could eat it again the next day. This punishment was repeated for eternity.

Prometheus was punished for his defiance of the gods and for stealing fire, a symbol of knowledge and power. This story can be used to illustrate the dangers of defying authority and the consequences of taking risks in the pursuit of knowledge.

Prometheus also represents the archetype of the trickster, who uses his wit and cunning to outsmart those in power. This archetype is often associated with creativity and innovation, as the trickster figure is willing to take risks and break rules in order to achieve their goals.

In a broader sense, the story of Prometheus can be seen as a cautionary tale about the dangers of playing with fire, both literally and metaphorically. It warns against the dangers of seeking knowledge